In [1]:
import pandas as pd
import joblib
import math


In [2]:
model = joblib.load("../models/load_model.pkl")


In [3]:
df = pd.read_csv("../data/support_tickets.csv", parse_dates=["created_at"])

daily = (
    df.groupby(df["created_at"].dt.date)
      .size()
      .reset_index(name="ticket_count")
)

daily.rename(columns={"created_at": "date"}, inplace=True)
daily["date"] = pd.to_datetime(daily["date"])

daily["day_of_week"] = daily["date"].dt.weekday
daily["is_weekend"] = daily["day_of_week"].isin([5, 6]).astype(int)
daily["lag_1"] = daily["ticket_count"].shift(1)
daily["lag_7"] = daily["ticket_count"].shift(7)
daily["rolling_7"] = daily["ticket_count"].rolling(7).mean()

daily = daily.dropna()


In [4]:
latest = daily.iloc[-1]

X_next = [[
    latest["day_of_week"],
    latest["is_weekend"],
    latest["lag_1"],
    latest["lag_7"],
    latest["rolling_7"]
]]


In [5]:
predicted_tickets = int(model.predict(X_next)[0])
predicted_tickets


C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(


120

In [6]:
TICKETS_PER_AGENT_PER_DAY = 20

agents_required = math.ceil(
    predicted_tickets / TICKETS_PER_AGENT_PER_DAY
)

agents_required


6

In [7]:
output = {
    "date": str(daily.iloc[-1]["date"].date()),
    "predicted_tickets": predicted_tickets,
    "agents_required": agents_required
}

output


{'date': '2024-06-28', 'predicted_tickets': 120, 'agents_required': 6}

In [8]:
future_predictions = []

temp_daily = daily.copy()

for i in range(7):
    latest = temp_daily.iloc[-1]

    X_next = [[
        latest["day_of_week"],
        latest["is_weekend"],
        latest["lag_1"],
        latest["lag_7"],
        latest["rolling_7"]
    ]]

    pred = int(model.predict(X_next)[0])

    future_predictions.append({
        "day": i + 1,
        "predicted_tickets": pred,
        "agents_required": math.ceil(pred / TICKETS_PER_AGENT_PER_DAY)
    })

    new_row = latest.copy()
    new_row["ticket_count"] = pred
    temp_daily = pd.concat([temp_daily, pd.DataFrame([new_row])], ignore_index=True)


C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\kgaje\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but SVR was 

In [9]:
pd.DataFrame(future_predictions)


,day,predicted_tickets,agents_required
0,1,120,6
1,2,120,6
2,3,120,6
3,4,120,6
4,5,120,6
5,6,120,6
6,7,120,6
